In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#  read the dataset
df = pd.read_csv("2012-sat-results.csv")

print(df.info())
print("")

# convert all values to numeric
df["SAT Critical Reading Avg. Score"] = pd.to_numeric(df["SAT Critical Reading Avg. Score"], errors="coerce")
df["SAT Math Avg. Score"] = pd.to_numeric(df["SAT Math Avg. Score"], errors="coerce")
df["SAT Writing Avg. Score"] = pd.to_numeric(df["SAT Writing Avg. Score"], errors="coerce")

# Drop rows with NaN values
df = df.dropna(subset=["SAT Critical Reading Avg. Score", "SAT Math Avg. Score", "SAT Writing Avg. Score"])

print(df.info())
print("")

# population params
mu = df["SAT Writing Avg. Score"].mean()
tao = df["SAT Writing Avg. Score"].sum()
sigmasq = df["SAT Writing Avg. Score"].var(ddof=0)

print(f"The mu is: {mu}")
print(f"The tao is: {tao}")
print(f"The sigma^2 is: {sigmasq}")

print("")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478 entries, 0 to 477
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   DBN                              478 non-null    object
 1   SCHOOL NAME                      478 non-null    object
 2   Num of SAT Test Takers           478 non-null    object
 3   SAT Critical Reading Avg. Score  478 non-null    object
 4   SAT Math Avg. Score              478 non-null    object
 5   SAT Writing Avg. Score           478 non-null    object
dtypes: object(6)
memory usage: 22.5+ KB
None

<class 'pandas.core.frame.DataFrame'>
Index: 421 entries, 0 to 477
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   DBN                              421 non-null    object 
 1   SCHOOL NAME                      421 non-null    object 
 2   Num of SA

# 1. Choose an auxiliary variable x that should be related to your variable of interest y. Take a SRS of size n (the same size as in Report 2)
Our variable of interest is SAT Writing Avg. Score. A related (auxilliary) variable we are using is SAT Math Avg. Score. The SAT Math Avg. Score has a 0.8885 correlation with SAT Writing Avg. Score.

In [3]:
# Correlation With an Auxilary Variable

df["SAT Math Avg. Score"].corr(df["SAT Writing Avg. Score"])

0.8884561176774858

In [4]:
n = 80
seed = 420
sampled_df = df.sample(n=n, replace=False, random_state=seed)

# 2. Perform a diagnostic analysis to determine if x and y have a linear relationship based on the sample data. Do regression analysis y ∼ x

In [6]:
import statsmodels.api as sm

X = sampled_df["SAT Math Avg. Score"]
y = sampled_df["SAT Writing Avg. Score"]

# Add a constant (intercept)
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X).fit()

# Get the summary
print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     SAT Writing Avg. Score   R-squared:                       0.858
Model:                                OLS   Adj. R-squared:                  0.857
Method:                     Least Squares   F-statistic:                     472.9
Date:                    Mon, 07 Apr 2025   Prob (F-statistic):           7.54e-35
Time:                            11:23:51   Log-Likelihood:                -362.46
No. Observations:                      80   AIC:                             728.9
Df Residuals:                          78   BIC:                             733.7
Df Model:                               1                                         
Covariance Type:                nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------

Since the p-value for the slope is 0 < 0.05, this means that you reject the null hypothesis that there is not a significant linear relationship between SAT math score and SAT writing score.

# 3. Based on the results of the regression analysis, make a conclusion about the appropriateness of using ratio and regression estimators.

Since the p-value of the intercept is 0.001 < 0.05, this means that you reject the null hypothesis and conclude the intercept is significantly different from 0. As seen in the previous part, there is a statistically significant linear linearship between SAT math score and SAT writing scores.

Both Ratio and Linear Regression estimators rely on the assumption that either μₓ or τₓ is known, which holds true for this data. However, Ratio estimators assume that the line has an intercept of 0, but this assumption is not met in this case. In contrast, the linear regression model does not require the intercept to be 0. Therefore, with this data, it is more appropriate to use the regression estimator rather than a ratio estimator.

# 4. Estimate your parameter of interest by Ratio estimator. Estimate its variance and give a confidence interval of α level chosen in Report 2.

##### Calculate $\hat \mu_r$

- $y =$ avg writing score
- $x =$ avg math score
- $\alpha = .05$
- Ratio Estimator of Population True Avg Writing Score:
- $\hat\mu_r = r * \mu_x$
    - where $mu_x$ is population mean of $x$'s
    - where $r$ is $\frac{\sum_{i=1}^n y_i}{\sum_{i=1}^n x_i} = \frac{\bar y}{\bar x}$

In [12]:
alpha = .05
mu_x = df["SAT Math Avg. Score"].mean()
r = sum(sampled_df["SAT Writing Avg. Score"])/sum(sampled_df["SAT Math Avg. Score"])
mu_r_hat = r * mu_x
print(f"mu_r_hat is: {mu_r_hat} \n") 

mu_r_hat is: 396.1233949575724 



##### Estimate variance of $\hat\mu_r$
- $\hat{\text{var}} (\hat\mu_r) = \frac{N-n}{N} * s^2_r / n$

- $s^2_r = \frac{1}{n-1}\sum_{i=1}^n (y_i - rx_i)^2$

In [14]:
N = df.shape[0]
s_r_squared = (1 / (n-1)) * sum((sampled_df["SAT Writing Avg. Score"] - r * sampled_df["SAT Math Avg. Score"])**2)
var_hat_mu_r_hat = ((N-n)/N) * s_r_squared / n
print(f"var_hat_mu_r_hat is: {var_hat_mu_r_hat} \n")

var_hat_mu_r_hat is: 6.04908217755827 



##### Confidence Interval
- $100(1-\alpha)\%$ CI for $\mu$ based on normal approx: $\hat\mu_r \pm t_{n-1,\frac{\alpha}{2}}\sqrt[]{\hat{\text{var}} (\hat\mu_r)}$

In [16]:
from scipy.stats import t

t_crit = t.ppf(1-(alpha/2), n-1)
lowerBound = mu_r_hat - t_crit * np.sqrt(var_hat_mu_r_hat)
upperBound = mu_r_hat + t_crit * np.sqrt(var_hat_mu_r_hat)
print(f"95% CI for mu_r_hat is: ({lowerBound}, {upperBound}) \n")


95% CI for mu_r_hat is: (391.22790616430814, 401.0188837508366) 



# 5. Estimate your parameter of interest by Regression estimator Estimate its variance and give a confidence interval of α level chosen in Report 2.

##### Calculate $\hat\mu_L$
- $\hat\mu_L = a + b*\mu_x$
- $a = \bar{y} - b * \bar{x}$
- $b = \frac{\sum_{i=1}^n (x_i - \bar{x})(y_i - \bar{y})}{\sum_{i=1}^n (x_i - \bar{x})^2}$

In [19]:
y_bar = sampled_df["SAT Writing Avg. Score"].mean()
x_bar = sampled_df["SAT Math Avg. Score"].mean()

bNumerator = sum((sampled_df["SAT Math Avg. Score"] - x_bar) * (sampled_df["SAT Writing Avg. Score"] - y_bar))
bDenominator = sum((sampled_df["SAT Math Avg. Score"] - x_bar)**2)
b = bNumerator / bDenominator

a = y_bar - b * x_bar

mu_L_hat = a + b * mu_x

print(f"mu_L_hat is: {mu_L_hat} \n")

mu_L_hat is: 395.4006470925351 



##### Estimate Variance of $\hat\mu_L$
- $\hat{\text{var}}(\hat{\mu_L}) = \frac{N-n}{Nn(n-2)}\sum_{i=1}^n(y_i - a - bx_i)^2$

In [21]:
var_hat_mu_L_hat = ((N - n) / (N * n * (n - 2))) * sum(((sampled_df["SAT Writing Avg. Score"] - a - b * sampled_df["SAT Math Avg. Score"])**2))
print(f"var_hat_mu_L_hat is: {var_hat_mu_L_hat} \n")

var_hat_mu_L_hat is: 5.239805102311415 



##### Confidence Interval
- 100(1-alpha)% CI: $\hat{\mu_L} \pm t_{n-2, \frac{\alpha}{2}}\sqrt[]{\hat{\text{var}}(\hat{\mu_L})}$

In [23]:
t_crit = t.ppf(1-(alpha/2), n-2)
lowerBound = mu_L_hat - t_crit * np.sqrt(var_hat_mu_L_hat)
upperBound = mu_L_hat + t_crit * np.sqrt(var_hat_mu_L_hat)
print(f"95% CI for mu_L_hat is: ({lowerBound}, {upperBound}) \n")

95% CI for mu_L_hat is: (390.8434746053701, 399.95781957970013) 



# 6. Choose the best estimator of your parameter based on estimated variance.

In [25]:
if var_hat_mu_r_hat > var_hat_mu_L_hat:
    print("var_hat_mu_L_hat is a better estimator of the paremeter as it has a lesser variance of", var_hat_mu_L_hat)
else:
    print("var_hat_mu_r_hat is a better estimator of the paremeter as it has a lesser variance of", var_hat_mu_r_hat)

var_hat_mu_L_hat is a better estimator of the paremeter as it has a lesser variance of 5.239805102311415


In this case, $\hat{\text{var}}(\hat{mu_L})$ is a better estimator of $\mu$ as it has a lower variance of 5.2398, compared to the variance of 6.049 given by the ratio estimator.

# 7. Calculate the true regression coefficients.
Namely, do regression $y ∼ x$ using whole data set (population). Is your conclusion in the part 3 changed? How does it change?

In [28]:
x = df["SAT Math Avg. Score"]
y = df["SAT Writing Avg. Score"]

# Add a constant (intercept)
x = sm.add_constant(x)

# Fit the model
model = sm.OLS(y, x).fit()

# Get the summary
print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     SAT Writing Avg. Score   R-squared:                       0.789
Model:                                OLS   Adj. R-squared:                  0.789
Method:                     Least Squares   F-statistic:                     1570.
Date:                    Mon, 07 Apr 2025   Prob (F-statistic):          8.43e-144
Time:                            11:23:51   Log-Likelihood:                -1983.0
No. Observations:                     421   AIC:                             3970.
Df Residuals:                         419   BIC:                             3978.
Df Model:                               1                                         
Covariance Type:                nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------

The true regression coefficients are $a = 61.0737$ and $b = 0.8054$. The $p$-values for the intercept and the slope are both $0$, thus we do not change our part $3$ conclusions. The assumption of linearity is met, however the regression line does not pass through $(0,0)$, thus the linear regression estimator is more appropriate to use than the ratio estimator.

# 8. Repeat steps 1-7 with another auxiliary variable

In [31]:
#  read the dataset
df = pd.read_csv("2012-sat-results.csv")

print(df.info())
print("")

# convert all values to numeric
df["SAT Critical Reading Avg. Score"] = pd.to_numeric(df["SAT Critical Reading Avg. Score"], errors="coerce")
df["SAT Math Avg. Score"] = pd.to_numeric(df["SAT Math Avg. Score"], errors="coerce")
df["SAT Writing Avg. Score"] = pd.to_numeric(df["SAT Writing Avg. Score"], errors="coerce")

# Drop rows with NaN values
df = df.dropna(subset=["SAT Critical Reading Avg. Score", "SAT Math Avg. Score", "SAT Writing Avg. Score"])

print(df.info())
print("")

# population params
mu = df["SAT Writing Avg. Score"].mean()
tao = df["SAT Writing Avg. Score"].sum()
sigmasq = df["SAT Writing Avg. Score"].var(ddof=0)

print(f"The mu is: {mu}")
print(f"The tao is: {tao}")
print(f"The sigma^2 is: {sigmasq}")

print("")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478 entries, 0 to 477
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   DBN                              478 non-null    object
 1   SCHOOL NAME                      478 non-null    object
 2   Num of SAT Test Takers           478 non-null    object
 3   SAT Critical Reading Avg. Score  478 non-null    object
 4   SAT Math Avg. Score              478 non-null    object
 5   SAT Writing Avg. Score           478 non-null    object
dtypes: object(6)
memory usage: 22.5+ KB
None

<class 'pandas.core.frame.DataFrame'>
Index: 421 entries, 0 to 477
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   DBN                              421 non-null    object 
 1   SCHOOL NAME                      421 non-null    object 
 2   Num of SA

In [32]:
# Our variable of interest is SAT Writing Avg. Score. A related (auxilliary) variable we are using is SAT Critical Reading Avg. Score. The SAT Critical Reading Avg. Score has a 
# 0.9703 correlation with SAT Writing Avg. Score.

df["SAT Critical Reading Avg. Score"].corr(df["SAT Writing Avg. Score"])

0.970342024638087

In [33]:
n = 80
seed = 8
sampled_df = df.sample(n=n, replace=False, random_state=seed)

In [34]:
import statsmodels.api as sm

X = sampled_df["SAT Critical Reading Avg. Score"]
y = sampled_df["SAT Writing Avg. Score"]

# Add a constant (intercept)
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X).fit()

# Get the summary
print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     SAT Writing Avg. Score   R-squared:                       0.975
Model:                                OLS   Adj. R-squared:                  0.975
Method:                     Least Squares   F-statistic:                     3101.
Date:                    Mon, 07 Apr 2025   Prob (F-statistic):           1.46e-64
Time:                            11:23:52   Log-Likelihood:                -319.35
No. Observations:                      80   AIC:                             642.7
Df Residuals:                          78   BIC:                             647.5
Df Model:                               1                                         
Covariance Type:                nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

Since the p-value for the slope is 0 < 0.05, this means that you reject the null hypothesis that there is not a significant linear relationship between SAT Critical Reading Avg. score and SAT writing Avg. score.

# Based on the results of the regression analysis, make a conclusion about the appropriateness of using ratio and regression estimators.

Since the p-value of the intercept is 0.032 < 0.05, this means that you reject the null hypothesis and conclude the intercept is significantly different from 0. As seen in the previous part, there is a statistically significant linear linearship between SAT math score and SAT writing scores.

Both Ratio and Linear Regression estimators rely on the assumption that either μₓ or τₓ is known, which holds true for this data. However, Ratio estimators assume that the line has an intercept of 0, but this assumption is not met in this case. In contrast, the linear regression model does not require the intercept to be 0. Therefore, with this data, it is more appropriate to use the regression estimator rather than a ratio estimator.

In [38]:
alpha = .05
mu_x = df["SAT Critical Reading Avg. Score"].mean()
r = sum(sampled_df["SAT Writing Avg. Score"])/sum(sampled_df["SAT Critical Reading Avg. Score"])
mu_r_hat = r * mu_x
print(f"mu_r_hat is: {mu_r_hat} \n") 

mu_r_hat is: 393.923955604779 



In [39]:
N = df.shape[0]
s_r_squared = (1 / (n-1)) * sum((sampled_df["SAT Writing Avg. Score"] - r * sampled_df["SAT Critical Reading Avg. Score"])**2)
var_hat_mu_r_hat = ((N-n)/N) * s_r_squared / n
print(f"var_hat_mu_r_hat is: {var_hat_mu_r_hat} \n")

var_hat_mu_r_hat is: 1.872166738592592 



In [40]:
from scipy.stats import t

t_crit = t.ppf(1-(alpha/2), n-1)
lowerBound = mu_r_hat - t_crit * np.sqrt(var_hat_mu_r_hat)
upperBound = mu_r_hat + t_crit * np.sqrt(var_hat_mu_r_hat)
print(f"95% CI for mu_r_hat is: ({lowerBound}, {upperBound}) \n")


95% CI for mu_r_hat is: (391.2004794256893, 396.6474317838687) 



In [41]:
y_bar = sampled_df["SAT Writing Avg. Score"].mean()
x_bar = sampled_df["SAT Critical Reading Avg. Score"].mean()

bNumerator = sum((sampled_df["SAT Critical Reading Avg. Score"] - x_bar) * (sampled_df["SAT Writing Avg. Score"] - y_bar))
bDenominator = sum((sampled_df["SAT Critical Reading Avg. Score"] - x_bar)**2)
b = bNumerator / bDenominator

a = y_bar - b * x_bar

mu_L_hat = a + b * mu_x

print(f"mu_L_hat is: {mu_L_hat} \n")


mu_L_hat is: 393.33610086294254 



In [42]:
var_hat_mu_L_hat = ((N - n) / (N * n * (n - 2))) * sum(((sampled_df["SAT Writing Avg. Score"] - a - b * sampled_df["SAT Critical Reading Avg. Score"])**2))
print(f"var_hat_mu_L_hat is: {var_hat_mu_L_hat} \n")

var_hat_mu_L_hat is: 1.7832087794495188 



In [43]:
t_crit = t.ppf(1-(alpha/2), n-2)
lowerBound = mu_L_hat - t_crit * np.sqrt(var_hat_mu_L_hat)
upperBound = mu_L_hat + t_crit * np.sqrt(var_hat_mu_L_hat)
print(f"95% CI for mu_L_hat is: ({lowerBound}, {upperBound}) \n")

95% CI for mu_L_hat is: (390.6775865862925, 395.9946151395926) 



In [44]:
if var_hat_mu_r_hat > var_hat_mu_L_hat:
    print("var_hat_mu_L_hat is a better estimator of the paremeter as it has a lesser variance of", var_hat_mu_L_hat)
else:
    print("var_hat_mu_r_hat is a better estimator of the paremeter as it has a lesser variance of", var_hat_mu_r_hat)


var_hat_mu_L_hat is a better estimator of the paremeter as it has a lesser variance of 1.7832087794495188


In [45]:
x = df["SAT Critical Reading Avg. Score"]
y = df["SAT Writing Avg. Score"]

# Add a constant (intercept)
x = sm.add_constant(x)

# Fit the model
model = sm.OLS(y, x).fit()

# Get the summary
print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     SAT Writing Avg. Score   R-squared:                       0.942
Model:                                OLS   Adj. R-squared:                  0.941
Method:                     Least Squares   F-statistic:                     6751.
Date:                    Mon, 07 Apr 2025   Prob (F-statistic):          1.67e-260
Time:                            11:23:52   Log-Likelihood:                -1713.1
No. Observations:                     421   AIC:                             3430.
Df Residuals:                         419   BIC:                             3438.
Df Model:                               1                                         
Covariance Type:                nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

# Calculate the true regression coefficients. Namely, do regression y ∼ x using whole data set (population). Is your conclusion in the part 3 changed? How does it change?

The true regression coefficients are $a = -7.5232 $ and $b = 1.0016$. The $p$-values for the intercept is 0.128 > 0.05, do not reject the null hypothesis. This means that it is plausible that the regression line passes through $(0, 0)$. The $p$-value of the intercept is 0 < 0.05, reject the null hypothesis. This means that we can conclude there is a linear relationship between SAT Critical Reading Avg. Score and SAT Writing Avg. Score. Both Ratio and Linear Regression estimators rely on the assumption that either μₓ or τₓ is known, which holds true for this data.

In part 3, we concluded one of the assumptions for Ratio Estimator was not met. This was the assumption that the regression line doesn't pass through $(0, 0)$. However, when using the whole population dataset, this assumption is met. Ratio estimator assumptions of μₓ or τₓ is known is met, passes through $(0, 0)$, and there is a linear relationship. Linear Regression estimator assumptions μₓ or τₓ is known is met, and there is a linear relationship. It would be appropriate to use either the Ratio Estimator or Linear Regression Estimator.

# 9. Based on estimation only, choose the variable that results in the most efficient estimator for parameter estimation.

I would choose SAT Critical Reading Avg. Score because it has lower variance for the ratio and regression estimators.

# 10. Based on estimation only, choose the estimator with the smallest estimated variance and ensure that the parameter falls within its confidence interval.

The estimator I will select is the regression estimator because it has smallest estimator of variance and mu is within the range (390.6776, 395.9946)